#### things to make modular
* query expansion
* query expansion result aggregation method
* using clean corpus
* retrieval method (dres/bm25)

#### imports

In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict
from corpus_expansion import expand_corpus
import copy

In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'
cosqa_deepseek_temp_0_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_queries_expanded_granite_temp_0pt5_mmr_2_path='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_granite_temp_0.5_mmr_2.csv'
cosqa_queries_variants_expanded_deepseek= '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/variants_based_prompting/cosqa_queries_variants_expanded_deepseek.csv'
cosqa_queries_variants_expanded_granite='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/variants_based_prompting/cosqa_queries_variants_expanded_granite.csv'
cosqa_granite_variants_corpus_expansions = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean_new_prompts.csv'
csn_queries_expanded_granite_temp_0pt5_mmr_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_temp_0.5_mmr.csv'
csn_deepseek_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_deepseek.csv'
csn_granite_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_granite.csv'
csn_code_based_queries_expanded = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_code_based.csv'
csn_test_queries_variants_expanded_granite = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/variants_based_prompting/csn_test_queries_variants_expanded_granite.csv'
csn_test_queries_variants_expanded_deepseek= '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/variants_based_prompting/csn_test_queries_variants_expanded_deepseek.csv'
csn_granite_test_variants_corpus_expansions = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_test_clean_new_prompts.csv'
csn_granite_train_variants_corpus_expansions= '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_train_clean_new_prompts.csv'
csn_granite_valid_variants_corpus_expansions = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_valid_clean_new_prompts.csv'

#### No QE

In [3]:
dataset_name = "cosqa" #CodeSearchNet-python  or cosqa
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

in tasks 
data loader init


0it [00:00, ?it/s]

In [4]:
df= pd.read_csv(cosqa_granite_updated_path) # csn_granite_path or cosqa_deepseek_updated_path
corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
tasks[dataset_name] = (corpus, queries, qrels)

#### Query Expansion

In [3]:
# CSN qe variants 
# csn_variants_qe_df = pd.read_csv(csn_test_queries_variants_expanded_deepseek)
# csn_variants_qe_df = csn_variants_qe_df.rename(columns= {'generated_explanation': 'expanded_query'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_variants_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)


# CSN code based qe
# csn_code_based_qe_df = pd.read_csv(csn_code_based_queries_expanded)
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'explanation': 'expanded_query'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'id': 'v_id'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'query_id': 'id'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_code_based_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# CSN
# csn_qe_df = pd.read_csv(csn_queries_expanded_granite_temp_0pt5_mmr_path)
# csn_qe_df=csn_qe_df.rename(columns={'mmr_explanation': 'expanded_query'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# COSQA
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

# expanded df
cosqa_qe_df = pd.read_csv(cosqa_queries_variants_expanded_deepseek)
cosqa_qe_df = cosqa_qe_df.rename(columns={'generated_explanation': 'expanded_query'})
cosqa_qe_df = cosqa_qe_df[cosqa_qe_df["id"].isin(queries.keys())]


# updating the corpus with no comments in code
# df= pd.read_csv(cosqa_granite_updated_path)
# new_corpus = update_corpus_with_cleaned_code(corpus, df) 

# updating the corpus with corpus expansions
df=pd.read_csv(cosqa_granite_variants_corpus_expansions) 
new_corpus = expand_corpus(corpus, df)

# updating queries for qe
new_queries, new_qrels = expand_queries_and_qrels(cosqa_qe_df, qrels)
tasks[dataset_name] = (new_corpus, new_queries, qrels)

in tasks 
data loader init


0it [00:00, ?it/s]

#### Explanations

In [4]:
# if you want to add explanations (deepseek / granite) (cosqa / csn)
tasks_expl = copy.deepcopy(tasks)
#add_expl(tasks_expl, dataset_name, csn_granite_path, "explanation_granite_1") # adding explanation instead of code
#add_expl(tasks, "CodeSearchNet-python", csn_deepseek_path, "explanation_deepseek_1") # adding explanation instead of code
add_expl(tasks_expl, "cosqa", cosqa_granite_updated_path, "explanation_granite_1_cleaned") # adding explanation instead of code

Total docs in corpus after replacement: 103020
Number of docs still missing 'text': 0


#### Running the pipeline

In [5]:
c,b,a = tasks[dataset_name]
f,e,d = tasks_expl[dataset_name]

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

run_params = [
    # run("intfloat/e5-base-v2", tasks, "baseline", "dres", "cosqa/query_corpus_expansions"),
    # run("BAAI/bge-base-en", tasks, "baseline", "dres", "cosqa/query_corpus_expansions"),
    run("intfloat/e5-base-v2", tasks, "granite1", "dres", "cosqa/query_corpus_expansions"),
    run("BAAI/bge-base-en", tasks, "granite1", "dres", "cosqa/query_corpus_expansions")
    
]

def run_wrapper(params):
    model_name, task, exp_type, retrieval, data_path = params
    return run(model_name, task, exp_type, retrieval, data_path)

# Run all in parallel
if __name__ == "__main__":
    max_workers = 4  # Adjust based on your machine's capacity

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_wrapper, params) for params in run_params]

        for future in as_completed(futures):
            try:
                result = future.result()
                print("Run completed successfully.")
            except Exception as e:
                print(f"Run failed: {e}")


YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  2500


Encoding batches:   0%|          | 0/10 [00:00<?, ?batch/s]

len of corpus:  103020
Encoding Corpus in batches... Warning: This might take a while!
Scoring Function: Cosine Similarity (cos_sim)
Encoding Batch 1/11...


Encoding batches:   0%|          | 0/40 [00:00<?, ?batch/s]

Encoding Batch 2/11...


...

Encoding Batch 10/11...


Encoding batches:   0%|          | 0/40 [00:00<?, ?batch/s]

Encoding Batch 11/11...


Encoding batches:   0%|          | 0/12 [00:00<?, ?batch/s]

exact_search finished
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by average
Retrieval evaluation results saved to results/cosqa/query_corpus_expansions/granite1/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.096, 'NDCG@3': 0.14728, 'NDCG@5': 0.17559, 'NDCG@10': 0.22598, 'NDCG@100': 0.31499, 'NDCG@1000': 0.3323}, 'MAP': {'MAP@1': 0.096, 'MAP@3': 0.134, 'MAP@5': 0.1494, 'MAP@10': 0.17015, 'MAP@100': 0.18736, 'MAP@1000': 0.18802}, 'Recall': {'Recall@1': 0.096, 'Recall@3': 0.186, 'Recall@5': 0.256, 'Recall@10': 0.412, 'Recall@100': 0.84, 'Recall@1000': 0.976}, 'Precision': {'P@1': 0.096, 'P@3': 0.062, 'P@5': 0.0512, 'P@10': 0.0412, 'P@100': 0.0084, 'P@1000': 0.00098}}}
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
